In [1]:
# import libraries
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import logging
from datetime import datetime

print('Starting...')
# Obtener la fecha y hora actual
fecha_hora_actual = datetime.now()

# Crear un manejador para la consola
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Formatear la fecha y hora
texto_fecha_hora = fecha_hora_actual.strftime("%Y%m%d_%H%M")
mylog = "Comebien_RAG_"+texto_fecha_hora+".log"

# Crear un manejador para el archivo
file_handler = logging.FileHandler(filename=mylog, encoding = "UTF-8")
file_handler.setLevel(logging.INFO)

# Definir el formato de los registros
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
#console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# Crear un logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Agregar los manejadores al logger
logger.addHandler(console_handler)
logger.addHandler(file_handler)
logger.info('Comienzo de script')

# ----- Data Indexing Process -----
# load your pdf doc
DOC_PATH = "./1080 Recetas de Cocina - Simone Ortega.pdf"
CHROMA_PATH = "RecetasDeLaAbuela" 
logger.info('Loading pdf ' + str(DOC_PATH))
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()
logger.info('End of load')

Comienzo de script
Loading pdf ./1080 Recetas de Cocina - Simone Ortega.pdf


Starting...


End of load


In [2]:
# split the doc into smaller chunks i.e. chunk_size=500
logger.info('Creating splitter... ')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1250, chunk_overlap=300)
logger.info('Splitting pdf... ')
chunks = text_splitter.split_documents(pages)

# get Embedding model
logger.info('Loading embeddings model... ')
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
#model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs={'device':'cuda'}, 
    encode_kwargs={'normalize_embeddings': True}
)
docs_after_split = chunks
sample_embedding = np.array(embeddings.embed_query(docs_after_split[0].page_content))
#print("Sample embedding of a document chunk: ", sample_embedding)
logger.info("Size of the embedding: ", str(sample_embedding.shape))

# embed the chunks as vectors and load them into the database
logger.info('Embedding chunks into chroma...')
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)
db_chroma.persist()
logger.info('End of Chroma embedding')

Creating splitter... 
Splitting pdf... 
Loading embeddings model... 
Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-l6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

--- Logging error ---
Traceback (most recent call last):
  File "/home/user/miniconda/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/home/user/miniconda/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/home/user/miniconda/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/home/user/miniconda/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/user/miniconda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/user/miniconda/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/user/miniconda/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/user/minicond

Batches:   0%|          | 0/85 [00:00<?, ?it/s]

End of Chroma embedding


In [3]:
# ----- Retrieval and Generation Process -----
# this is an example of a user question (query)
#query = 'what are the top risks mentioned in the document?'
#query = 'asado de cordero'
#query = 'salmorejo'
#query = 'Lentejas con chorizo'
#query = 'Paella valenciana'
query = 'Cocido de garbanzos'
topK = 2

# retrieve context - top 5 most relevant (closests) chunks to the query vector 
# (by default Langchain is using cosine distance metric)
logger.info('Retrieving context from chroma: '+str(query))
docs_chroma = db_chroma.similarity_search_with_score(query, k=topK)

# generate an answer based on given user query and retrieved context information
retriever1 = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

for doc, _score in docs_chroma:
    print('Doc: '+str(doc))
    print('Page: '+str(doc.page_content))
    print('Score: '+str(_score))
    

Retrieving context from chroma: Cocido de garbanzos


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Doc: page_content='Garbanzos\n. Para que los garbanzos\nsalgan tiernos hay que ponerlos a cocer' metadata={'page': 150, 'source': './1080 Recetas de Cocina - Simone Ortega.pdf'}
Page: Garbanzos
. Para que los garbanzos
salgan tiernos hay que ponerlos a cocer
Score: 0.6452703475952148
Doc: page_content='en trozos y la morcilla en rodajas) y la\nmitad de los garbanzos. Seguidamente se' metadata={'page': 543, 'source': './1080 Recetas de Cocina - Simone Ortega.pdf'}
Page: en trozos y la morcilla en rodajas) y la
mitad de los garbanzos. Seguidamente se
Score: 0.6575905084609985


In [4]:
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
vectorstore = FAISS.from_documents(docs_after_split, embeddings)
#query = 'asado de cordero'
topK = 1
# Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search_with_score(query, k=topK)

logger.info('There are ' +str(len(relevant_documents)) + ' documents retrieved which are relevant to the query. Display the first one:')
for doc, _score in relevant_documents:
    texto = str(doc.page_content)
    texto = texto.replace('\n', ' ')
    texto = texto.replace('\r', ' ')
    logger.info('Doc: '+str(texto))
    logger.info('Score: '+str(_score))

# Use similarity searching algorithm and return most relevant documents.
logger.info('Recovering as retriever...')
retriever2 = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": topK})


Batches:   0%|          | 0/85 [00:00<?, ?it/s]

Loading faiss with AVX512 support.
Successfully loaded faiss with AVX512 support.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

There are 1 documents retrieved which are relevant to the query. Display the first one:
Doc: también. Nota.-EI caldo de cocer los garbanzos es muy bueno y se puede utilizar para cocer arroz, hacer una sopa, etc. 191.—GARBANZOS REFRITOS (6 personas) ½ kg. de garbanzos, 100 gr. de manteca de cerdo, 1 cebolla mediana (100 gr.), 3 tomates medianos bien maduros, 1 cucharada (de las de café) de pimentón, ½ chorizo de cantimpalos, agua y sal. Se ponen los garbanzos en remojo la víspera (o unas 12 horas antes) en agua
Score: 0.7616598
Recovering as retriever...


In [6]:
#llm_model = 'xai-org/grok-1'
#llm_model = 'unsloth/mistral-7b-instruct-v0.2-bnb-4bit'
llm_model = "mistralai/Mistral-7B-v0.1"
llm_model = 'google/gemma-7b'
llm_model = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
llm_model = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
llm_model = 'google/gemma-2b'
llm_model = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
uns_model = 'unsloth/gemma-2b-bnb-4bit'
sft_model = 'somosnlp/RecetasDeLaAbuela5k_gemma-2b-bnb-4bit'

hf_token = 'xxx_key'
logger.info('Llamando al modelo '+str(llm_model))
query = 'asado de cordero'  # Sample question, change to other questions you are interested in.
max_seq_length = 500

hf_hub_enable = False
if hf_hub_enable:
    #from langchain_community.llms import HuggingFaceHub
    from langchain_community.llms import HuggingFaceEndpoint
    #hf = HuggingFaceHub(repo_id=llm_model, huggingfacehub_api_token=hf_token , model_kwargs={"temperature":0.1, "max_length":500})
    #hf = HuggingFaceHub(repo_id=llm_model, huggingfacehub_api_token=hf_token , task="text-generation", model_kwargs={"temperature":0.1, "max_new_tokens":250, "repetition_penalty": 1.3, })
    # Basic Example (no streaming)        
    hf = HuggingFaceEndpoint(repo_id=llm_model,
    max_new_tokens=max_seq_length, top_k=10, top_p=0.95, typical_p=0.95, temperature=0.1, repetition_penalty=1.2, huggingfacehub_api_token=hf_token)
    prompt_template = """Responde según el contexto a la pregunta final de forma concisa. No digas lo que no esta. No cites recetas del contexto. Cocina castellana. Responde la pregunta con: 1) Nombre, 2) Ingredientes, 3) Receta paso a paso. Contexto: {context} Pregunta: {question} Respuesta: """
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])    
else:
    !pip install torch sentencepiece
    import torch
    major_version, minor_version = torch.cuda.get_device_capability()
    # Must install separately since Colab has torch 2.2.1, which breaks packages
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    if major_version >= 8:
        # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
        !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
    else:
        # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
        print('Installing dependencies older GPUs...')
        !pip install --no-deps xformers trl peft accelerate bitsandbytes
    pass

    if False:
        from unsloth import FastLanguageModel   
        from langchain.chains import LLMChain
        !pip install langchain --upgrade
        model, tokenizer = FastLanguageModel.from_pretrained(model_name = llm_model, max_seq_length = max_seq_length, dtype=None, load_in_4bit = True,)
        prompt_template = """Responde según el contexto a la pregunta final de forma concisa. No digas lo que no esta. No cites recetas del contexto. Cocina castellana. Responde la pregunta con: 1) Nombre, 2) Ingredientes, 3) Receta paso a paso. Contexto: {context} Pregunta: {question} Respuesta: """
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
        hf = LLMChain(prompt=prompt, llm=model)
    else:
        gemma_enable = True
        if gemma_enable:
            from unsloth import FastLanguageModel
            model, tokenizer = FastLanguageModel.from_pretrained(model_name = sft_model, max_seq_length = max_seq_length,dtype=None, load_in_4bit = True,)
        else:
            from unsloth import FastLlamaModel, FastMistralModel
            from peft import PeftModel
            base_model, tokenizer = FastMistralModel.from_pretrained(model_name = uns_model, max_seq_length = max_seq_length,dtype = None,load_in_4bit = True,)
            logger.info('Modelo base cargado ' +str(uns_model))
            ft_model = PeftModel.from_pretrained(base_model, sft_model, torch_dtype=torch.float16,)
            logger.info(type(ft_model))
            logger.info('Mezclando modelos...')
            model = ft_model.merge_and_unload()
        model.save_pretrained('./model')
        tokenizer.save_pretrained('./model')

if hf_hub_enable:
    
    hf.invoke(query)
    logger.info('Preguntando al modelo...'+str(llm_model))
    retrievalQA = RetrievalQA.from_chain_type(
        llm=hf,
        chain_type="stuff",
        retriever=retriever2,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )

    # Call the QA chain with our query.
    logger.info('Recuperando respuesta de cadena QA...'+str(llm_model))
    result = retrievalQA.invoke({"query": query})
    logger.info(result['result'])

    if False:
        relevant_docs = result['source_documents']
        logger.info(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

        for i, doc in enumerate(relevant_docs):
            logger.info(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
            logger.info(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

logger.info('End of script')


Llamando al modelo mistralai/Mixtral-8x7B-Instruct-v0.1
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-_u22i_ho/unsloth_98720b268bff4b3388c2f68b5bdf228c
  Running command git clone -q https://github.com/unslothai/unsloth.git /tmp/pip-install-_u22i_ho/unsloth_98720b268bff4b3388c2f68b5bdf228c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
Installing dependencies older GPUs...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


==((====))==  Unsloth: Fast Gemma patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.581 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


PyTorch version 2.2.2 available.
Unsloth 2024.4 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.
End of script


In [7]:
from transformers import  StoppingCriteria, StoppingCriteriaList, GenerationConfig

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
class ListOfTokensStoppingCriteria(StoppingCriteria):
    """
    Clase para definir un criterio de parada basado en una lista de tokens específicos.
    """
    def __init__(self, tokenizer, stop_tokens):
        self.tokenizer = tokenizer
        # Codifica cada token de parada y guarda sus IDs en una lista
        self.stop_token_ids_list = [tokenizer.encode(stop_token, add_special_tokens=False) for stop_token in stop_tokens]

    def __call__(self, input_ids, scores, **kwargs):
        # Verifica si los últimos tokens generados coinciden con alguno de los conjuntos de tokens de parada
        for stop_token_ids in self.stop_token_ids_list:
            len_stop_tokens = len(stop_token_ids)
            if len(input_ids[0]) >= len_stop_tokens:
                if input_ids[0, -len_stop_tokens:].tolist() == stop_token_ids:
                    return True
        return False

# Uso del criterio de parada personalizado
stop_tokens = ["<end_of_turn>"]  # Lista de tokens de parada

# Inicializa tu criterio de parada con el tokenizer y la lista de tokens de parada
stopping_criteria = ListOfTokensStoppingCriteria(tokenizer, stop_tokens)

# Añade tu criterio de parada a una StoppingCriteriaList
stopping_criteria_list = StoppingCriteriaList([stopping_criteria])

def generate_text(prompt, context, model, max_length=max_seq_length):
  prompt=prompt.replace("\n", "").replace("¿","").replace("?","")
  enable_gemma = True
  if enable_gemma:
    input_text = f'''<bos><start_of_turn>system\n{context}<end_of_turn><start_of_turn>user\n{prompt}<end_of_turn><start_of_turn>model\n'''
    logger.info(input_text)
  else:
    intro = 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request'
    input_text = f'''{intro}\n\n### Instruction:\n\n{context}\n\n### Input:\n{prompt}\n\n### Response: '''
  inputs = tokenizer.encode(input_text, return_tensors="pt", add_special_tokens=False).to("cuda:0")
  max_new_tokens=max_length
  generation_config = GenerationConfig(max_new_tokens=max_new_tokens, temperature=0.32, repetition_penalty=1.3, do_sample=True,)
  if enable_gemma:
    outputs = model.generate(generation_config=generation_config, input_ids=inputs, stopping_criteria=stopping_criteria_list,)
  else:
    outputs = model.generate(generation_config=generation_config, input_ids=inputs)
  return tokenizer.decode(outputs[0], skip_special_tokens=False) #True

def mostrar_respuesta(pregunta, contexto, logger, model):
    try:
        res= generate_text(pregunta, contexto, model, 700)
    except Exception as e:
        res = e    
    logger.info(str(res))
    return res


In [12]:
test1 = True
if test1:
    context = 'Eres un agente experto en nutrición y cocina. '
    query = 'Dime los ingredientes y pasos de la receta '
    recetas = ["Tortilla de patatas española", "Ceviche", "Enchiladas", "Empanada de puerro y nata", "Cordero asado", "Paella", "Guiso de carne", "Potaje de garbanzos", "Callos madrileños" ]
    for receta in recetas:        
        # retrieve context - most relevant (closests) chunks to the query vector (by default Langchain is using cosine distance metric)
        enable_chroma = False
        if enable_chroma:
            docs = db_chroma.similarity_search_with_score(receta, k=1)
        else:
            docs = vectorstore.similarity_search_with_score(receta, k=1)            
        contexto = docs[0][0].page_content 
        contexto = context + contexto.replace("\n", " ")           
        receta = query + receta
        logger.info('Contexto '+str(contexto)+' Pregunta: '+str(receta))
        mostrar_respuesta(receta, contexto, logger, model)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Contexto Eres un agente experto en nutrición y cocina. Gelatina de naranja Fruta Jueves Tortilla de patatas Guisantes sencillos Rabillo de cadera guisado con zanahorias y cebollitas Salchichas con puré de patatas Fruta Queso de Burgos Viernes Alioli: patatas, zanahorias, alcachofas y bacalao cocido Zanahorias en salsa Huevos en Pregunta: Dime los ingredientes y pasos de la receta Tortilla de patatas española
<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina. Gelatina de naranja Fruta Jueves Tortilla de patatas Guisantes sencillos Rabillo de cadera guisado con zanahorias y cebollitas Salchichas con puré de patatas Fruta Queso de Burgos Viernes Alioli: patatas, zanahorias, alcachofas y bacalao cocido Zanahorias en salsa Huevos en<end_of_turn><start_of_turn>user
Dime los ingredientes y pasos de la receta Tortilla de patatas española<end_of_turn><start_of_turn>model

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina. Gelatina de naranja Fruta Jue

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Contexto Eres un agente experto en nutrición y cocina. Repollo con patatas y mayonesa Coles de Bruselas con bechamel Redondo guisado Fiambres variados Macedonia de frutas Compota de manzanas Miércoles Arroz con tomate, salchichas, pimientos y guisantes Acelgas con tomate Resto de redondo con bechamel y alcaparras Filetes de castañola al horno Natillas Fruta Jueves Pregunta: Dime los ingredientes y pasos de la receta Ceviche
<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina. Repollo con patatas y mayonesa Coles de Bruselas con bechamel Redondo guisado Fiambres variados Macedonia de frutas Compota de manzanas Miércoles Arroz con tomate, salchichas, pimientos y guisantes Acelgas con tomate Resto de redondo con bechamel y alcaparras Filetes de castañola al horno Natillas Fruta Jueves<end_of_turn><start_of_turn>user
Dime los ingredientes y pasos de la receta Ceviche<end_of_turn><start_of_turn>model

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocin

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Contexto Eres un agente experto en nutrición y cocina. picada con la carne,   125 gr. de miga de pan (del día anterior),   125 gr. de champiñones de París frescos,   2 pata grande de ternera en trozos, chamuscada y lavada, aromáticas (o un ramillete con laurel, tomillo, perejil y ajo),   1 vaso (de los de agua) no lleno de leche hirviendo, agua fría,   sal y pimienta negra molida. Se pone la miga de pan en remojo con la leche hirviendo, y mientras tanto se limpian y se lavan muy bien los champiñones; se secan con un trapo limpio y se pican bastante menudos. En una ensaladera se pone la carne Pregunta: Dime los ingredientes y pasos de la receta Guiso de carne
<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina. picada con la carne,   125 gr. de miga de pan (del día anterior),   125 gr. de champiñones de París frescos,   2 pata grande de ternera en trozos, chamuscada y lavada, aromáticas (o un ramillete con laurel, tomillo, perejil y ajo),   1 vaso (de los de agua) no

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Contexto Eres un agente experto en nutrición y cocina. Compota de manzanas Queso Jueves Arroz al curry Alcachofas con vinagreta Pierna de cordero asada, con ensalada Criadillas empanadas, con arroz blanco Fruta Zumo de naranja Viernes Potaje de garbanzos con espinacas y bacalao Sopa fina de tapioca Tortilla de patatas Conchas de gambas Pregunta: Dime los ingredientes y pasos de la receta Potaje de garbanzos
<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina. Compota de manzanas Queso Jueves Arroz al curry Alcachofas con vinagreta Pierna de cordero asada, con ensalada Criadillas empanadas, con arroz blanco Fruta Zumo de naranja Viernes Potaje de garbanzos con espinacas y bacalao Sopa fina de tapioca Tortilla de patatas Conchas de gambas<end_of_turn><start_of_turn>user
Dime los ingredientes y pasos de la receta Potaje de garbanzos<end_of_turn><start_of_turn>model

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina. Compota de manzanas Queso Jueve

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Contexto Eres un agente experto en nutrición y cocina. Perdices. Gambas. Repollo. Conejo de monte. Chirlas. Apio. Liebre. Langostinos. Nabos. Faisán. Langosta. Alcachofas. Becada. Bogavante. Calabaza. Codornices Carabineros. Berros. Aves acuáticas. Quisquillas. Jabalí. Almejas. Venado Mejillones. Angulas. Ostras. FEBRERO Pregunta: Dime los ingredientes y pasos de la receta Callos madrileños
<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina. Perdices. Gambas. Repollo. Conejo de monte. Chirlas. Apio. Liebre. Langostinos. Nabos. Faisán. Langosta. Alcachofas. Becada. Bogavante. Calabaza. Codornices Carabineros. Berros. Aves acuáticas. Quisquillas. Jabalí. Almejas. Venado Mejillones. Angulas. Ostras. FEBRERO<end_of_turn><start_of_turn>user
Dime los ingredientes y pasos de la receta Callos madrileños<end_of_turn><start_of_turn>model

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina. Perdices. Gambas. Repollo. Conejo de monte. Chirlas. Apio. Liebre

In [9]:
print('End')

End
